In [78]:
import kagglehub

path = kagglehub.dataset_download("shenba/time-series-datasets")

print("Path to dataset files:", path)

Path to dataset files: /home/hitmonlee/.cache/kagglehub/datasets/shenba/time-series-datasets/versions/1


# CluStream

In [79]:
import pandas as pd

df = pd.read_csv(filepath_or_buffer=path + "/" + "Electric_Production.csv")

In [80]:
data = df["IPG2211A2N"]
data.index = pd.to_datetime(df["DATE"])
data


DATE
1985-01-01     72.5052
1985-02-01     70.6720
1985-03-01     62.4502
1985-04-01     57.4714
1985-05-01     55.3151
                ...   
2017-09-01     98.6154
2017-10-01     93.6137
2017-11-01     97.3359
2017-12-01    114.7212
2018-01-01    129.4048
Name: IPG2211A2N, Length: 397, dtype: float64

In [81]:
from river import cluster

clustream = cluster.CluStream(
    n_macro_clusters=5,
    max_micro_clusters=15,
    time_gap=1,  
    seed=42,
)

In [82]:
for dt, val in data.items():
    x = {
        "value": val,
        "dayofweek": dt.dayofweek,
        "month": dt.month,
        "day": dt.day
    }
    cluster_id = clustream.learn_one(x)
    print(dt, cluster_id)


1985-01-01 00:00:00 None
1985-02-01 00:00:00 None
1985-03-01 00:00:00 None
1985-04-01 00:00:00 None
1985-05-01 00:00:00 None
1985-06-01 00:00:00 None
1985-07-01 00:00:00 None
1985-08-01 00:00:00 None
1985-09-01 00:00:00 None
1985-10-01 00:00:00 None
1985-11-01 00:00:00 None
1985-12-01 00:00:00 None
1986-01-01 00:00:00 None
1986-02-01 00:00:00 None
1986-03-01 00:00:00 None
1986-04-01 00:00:00 None
1986-05-01 00:00:00 None
1986-06-01 00:00:00 None


1986-07-01 00:00:00 None
1986-08-01 00:00:00 None
1986-09-01 00:00:00 None
1986-10-01 00:00:00 None
1986-11-01 00:00:00 None
1986-12-01 00:00:00 None
1987-01-01 00:00:00 None
1987-02-01 00:00:00 None
1987-03-01 00:00:00 None
1987-04-01 00:00:00 None
1987-05-01 00:00:00 None
1987-06-01 00:00:00 None
1987-07-01 00:00:00 None
1987-08-01 00:00:00 None
1987-09-01 00:00:00 None
1987-10-01 00:00:00 None
1987-11-01 00:00:00 None
1987-12-01 00:00:00 None
1988-01-01 00:00:00 None
1988-02-01 00:00:00 None
1988-03-01 00:00:00 None
1988-04-01 00:00:00 None
1988-05-01 00:00:00 None
1988-06-01 00:00:00 None
1988-07-01 00:00:00 None
1988-08-01 00:00:00 None
1988-09-01 00:00:00 None
1988-10-01 00:00:00 None
1988-11-01 00:00:00 None
1988-12-01 00:00:00 None
1989-01-01 00:00:00 None
1989-02-01 00:00:00 None
1989-03-01 00:00:00 None
1989-04-01 00:00:00 None
1989-05-01 00:00:00 None
1989-06-01 00:00:00 None
1989-07-01 00:00:00 None
1989-08-01 00:00:00 None
1989-09-01 00:00:00 None
1989-10-01 00:00:00 None


# Kmeans

In [88]:
micro_centers = [mc.center for mc in clustream.micro_clusters.values()]

micro_centers = [[cen for cen in cens.values()] for cens in micro_centers]
micro_centers


[[126.82985, 1.0, 1.0, 1.0],
 [113.1213, 4.5, 12.0, 1.0],
 [114.7212, 4.0, 12.0, 1.0],
 [92.40626000000003, 2.975, 10.3, 1.0],
 [117.42747499999999, 3.9999999999999996, 1.1250000000000002, 1.0],
 [106.59922727272728, 2.1818181818181817, 12.0, 1.0],
 [89.847521875, 2.9375, 3.65625, 1.0],
 [85.09011249999999, 2.75, 9.624999999999998, 1.0],
 [75.28121538461538, 3.307692307692308, 9.23076923076923, 1.0],
 [79.83623103448275, 3.172413793103449, 4.689655172413795, 1.0],
 [100.87271846153845, 3.1538461538461537, 5.723076923076923, 1.0],
 [110.83577249999999, 2.775, 5.375, 1.0],
 [73.68946000000001, 2.4, 2.933333333333333, 1.0],
 [67.5809380952381, 2.857142857142857, 7.119047619047619, 1.0],
 [59.493700000000004, 3.7, 6.8, 1.0]]

In [89]:
from sklearn.cluster import KMeans
import numpy as np

X = np.array(micro_centers)

kmeans = KMeans(n_clusters=3,random_state=42)
macro_labels = kmeans.fit_predict(X)

macro_labels

array([2, 0, 0, 0, 2, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1], dtype=int32)